## This code illustrates how to get S&P500 index constituents and their identifiers from CRSP and Compustat
- Researchers used to be able to extract index membership information from Compustat's "comp.idxcst_his" data. Now that S&P pulled this piece of data off from WRDS platform, we have to turn to CRSP to get S&P500 Index membership data. 
- Unfortunately, there is no easy way to uncover constituents info for the other indices covered by "comp.idxcst_his".

In [1]:
##########################################
# S&P 500 Index Constituents             #
# Qingyi (Freda) Song Drechsler          #
# Date: October 2020                     #
##########################################

import pandas as pd
import wrds

### Step 1: Connect to WRDS

In [2]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [3]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/12/1989'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [4]:
sp500.head()

,permno,start,ending,date,ret
0,24678,1957-03-01,2000-06-20,1989-01-31,0.039872
1,17750,1957-03-01,2023-12-29,1989-01-31,0.096567
2,66114,1986-12-18,2007-12-28,1989-01-31,0.146982
3,38914,1969-05-22,2001-03-30,1989-01-31,0.031847
4,19553,1957-03-01,1998-12-31,1989-01-31,0.036667


In [5]:
sp500.date.sort_values().unique()

<DatetimeArray>
['1989-01-31 00:00:00', '1989-02-28 00:00:00', '1989-03-31 00:00:00',
 '1989-04-28 00:00:00', '1989-05-31 00:00:00', '1989-06-30 00:00:00',
 '1989-07-31 00:00:00', '1989-08-31 00:00:00', '1989-09-29 00:00:00',
 '1989-10-31 00:00:00',
 ...
 '2023-03-31 00:00:00', '2023-04-28 00:00:00', '2023-05-31 00:00:00',
 '2023-06-30 00:00:00', '2023-07-31 00:00:00', '2023-08-31 00:00:00',
 '2023-09-29 00:00:00', '2023-10-31 00:00:00', '2023-11-30 00:00:00',
 '2023-12-29 00:00:00']
Length: 420, dtype: datetime64[ns]

### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
- You don't need this step if only PERMNO is required
- This step aims to add TICKER, SHRCD, EXCHCD and etc. 

In [6]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

In [7]:
mse.sample(5)

,comnam,ncusip,namedt,nameendt,permno,shrcd,exchcd,hsiccd,ticker
37845,EMETT & CHANDLER COS INC,29120510,1982-11-01,1986-05-15,32396,10,3,6411,EMCC
73906,DEPARTMENT 56 INC,24950910,2002-01-02,2004-06-09,79328,11,1,5023,DFS
53639,WESTERN PACIFIC RR CO DE,95913010,1980-04-07,1982-10-31,61073,10,3,4011,WPRRA
59041,DIVI HOTELS N V,25537410,1986-09-05,1991-11-27,69913,12,2,7011,DVH
70416,TRINITY BIOTECH PLC,89643810,1994-10-24,2000-10-31,78002,31,3,2835,TRIBY


In [8]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker
693271,85926,1998-04-01,2023-12-15,2005-03-31,-0.006313,SEALED AIR CORP NEW,81211K10,2004-06-10,2012-05-03,11,1,2673,SEE
303890,59010,1986-08-21,2022-02-02,1996-06-28,-0.042379,GAP INC,36476010,1985-06-03,2002-01-01,11,1,5651,GPS
889582,24766,1985-06-13,2023-12-29,2009-06-30,-0.040739,NORTHROP GRUMMAN CORP,66680710,2004-06-10,2023-12-29,11,1,3812,NOC
226695,20394,1973-03-01,1996-09-30,1994-07-29,0.125000,OUTBOARD MARINE CORP,69002010,1968-01-02,1997-09-30,11,1,3519,OM
1244235,77129,2006-04-03,2023-12-29,2017-04-28,-0.069262,KIMCO REALTY CORP,49446R10,2010-12-03,2019-09-11,18,1,6798,KIM


### Step 4: Add Compustat Identifiers
- Link with Compustat's GVKEY and IID if need to work with fundamental data
- Linkage is done through crsp.ccmxpf_linktable

In [9]:
# Linking with Compustat through CCM

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [10]:
# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]
sp500ccm.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker,gvkey,iid,linktype,linkprim,linkdt,linkenddt
196246,38156,1975-02-06,2023-12-29,2012-10-31,0.000572,WILLIAMS COS,96945710,2004-06-10,2023-12-29,11,1,4922,WMB,011506,01,LU,C,1962-07-02,2024-05-14 11:37:18.952446
194336,81655,1995-05-30,2023-12-29,2012-07-31,0.020739,DARDEN RESTAURANTS INC,23719410,2004-06-10,2014-07-20,11,1,5812,DRI,031846,01,LC,P,1995-05-30,2024-05-14 11:37:18.952446
142668,14656,1969-03-06,2012-10-04,2005-09-30,0.075653,SUNOCO INC,86764P10,2004-06-10,2012-10-04,11,1,2911,SUN,010156,01,LU,P,1962-01-31,2024-05-14 11:37:18.952446
145764,49373,1986-11-28,2020-09-18,2006-02-28,-0.088307,BLOCK H & R INC,09367110,2004-06-10,2006-06-29,11,1,7291,HRB,002269,01,LU,P,1969-10-13,2024-05-14 11:37:18.952446
243541,23473,1997-12-18,2023-12-29,2019-02-28,0.070266,CINCINNATI FINANCIAL CORP,17206210,2004-06-10,2021-03-30,11,3,6330,CINF,014824,01,LC,P,1982-01-29,2024-05-14 11:37:18.952446


In [11]:
# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]
sp500ccm.sample(5)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
109847,2001-08-31,22066,NATIONAL SERVICE INDUSTRIES INC,63765710,11,1,3646,NSI,007773,01,1986-12-04,2001-11-30,0.043325
153686,2007-03-30,10145,HONEYWELL INTERNATIONAL INC,43851610,11,1,3724,HON,001300,01,1925-12-31,2023-12-29,-0.007328
270155,2022-10-31,23660,CINTAS CORP,17290810,11,3,2320,CTAS,003062,01,2001-03-01,2023-12-29,0.101394
15065,1990-09-28,18112,FOSTER WHEELER CORP,35024410,11,1,1600,FWC,004864,01,1957-03-01,2000-01-28,0.078788
190691,2012-01-31,40125,COMPUTER SCIENCES CORP,20536310,11,1,7373,CSC,003336,01,1981-05-14,2015-11-30,0.089873


In [12]:
cnt = sp500ccm.groupby(['date'])['permno'].count().reset_index().rename(columns={'permno':'npermno'})
cnt.sample(4)

,date,npermno
289,2013-02-28,501
169,2003-02-28,500
297,2013-10-31,501
254,2010-03-31,501


In [13]:
sp500ccm.sample(10)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
194755,2012-07-31,77730,TYSON FOODS INC,90249410,11,1,2015,TSN,010793,01,2005-08-11,2023-12-29,-0.202868
13775,1990-07-31,36469,FIRST UNION CORP,33735810,11,1,6021,FTU,004739,01,1989-02-09,2008-12-31,-0.066225
211480,2014-10-31,66093,A T & T INC,00206R10,11,1,4812,T,009899,01,1983-12-01,2023-12-29,0.001703
146461,2006-03-31,11308,COCA COLA CO,19121610,11,1,2086,KO,003144,01,1957-03-01,2023-12-29,0.005004
78414,1997-11-28,21186,WESTVACO CORP,96154810,11,1,2631,W,011446,01,1957-03-01,2023-12-29,-0.005714
209040,2014-06-30,35051,PALL CORP,69642930,11,1,3569,PLL,008304,01,1987-12-31,2015-08-28,0.007671
74542,1997-05-30,75181,PLACER DOME INC,72590610,12,1,1041,PDG,008626,01,1987-08-20,2002-07-19,0.119084
54600,1995-02-28,21004,G T E CORP,36232010,11,1,4813,GTE,004961,01,1957-03-01,2000-06-30,-0.000886
18163,1991-01-31,21573,INTERNATIONAL PAPER CO,46014610,11,1,2621,IP,006104,01,1925-12-31,2023-12-29,0.100467
164507,2008-07-31,41355,PARKER HANNIFIN CORP,70109410,11,1,3593,PH,008358,01,1985-11-07,2023-12-29,-0.135165


In [14]:
sp500ccm.to_parquet('./data/stocks_data/stocks_list/SP500_full_original_data.parquet')